# steam_games

In [1]:
from extract import jsongz

#df_steam_game_original = jsongz.leer_archivo("..\\data\\raw\\steam_games.json.gz")
df_original = jsongz.leer_archivo("../data/raw/steam_games.json.gz")

In [2]:
df = df_original.copy()
df.head()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df.shape[0]

120445

El df contiene 120445 registros

Evaluar los tipos de datos usados por las columnas

In [4]:
from transform import info

info.mostrar_info_key_types(df)

publisher - {<class 'str'>, <class 'float'>}
genres - {<class 'float'>, <class 'list'>}
app_name - {<class 'str'>, <class 'float'>}
title - {<class 'str'>, <class 'float'>}
url - {<class 'str'>, <class 'float'>}
release_date - {<class 'str'>, <class 'float'>}
tags - {<class 'float'>, <class 'list'>}
reviews_url - {<class 'str'>, <class 'float'>}
specs - {<class 'float'>, <class 'list'>}
price - {<class 'str'>, <class 'float'>}
early_access - {<class 'bool'>, <class 'float'>}
id - {<class 'str'>, <class 'float'>}
developer - {<class 'str'>, <class 'float'>}


Recordar que las columnas que contiene valores iguales a "float", podria indica que contine valores nulos.

Las columnas que utilizan el tipo de dato "list" deben ser desanidados

In [5]:
from transform import normalizar

df = normalizar.desanidar_dataframe(df, ["genres"])
df.head()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Volver a revisar los tipos de datos, para asegurar que no se utilizen estructuras de datos

In [6]:
from transform import info

info.mostrar_info_key_types(df)

publisher - {<class 'str'>, <class 'float'>}
genres - {<class 'str'>, <class 'float'>}
app_name - {<class 'str'>, <class 'float'>}
title - {<class 'str'>, <class 'float'>}
url - {<class 'str'>, <class 'float'>}
release_date - {<class 'str'>, <class 'float'>}
tags - {<class 'float'>, <class 'list'>}
reviews_url - {<class 'str'>, <class 'float'>}
specs - {<class 'float'>, <class 'list'>}
price - {<class 'str'>, <class 'float'>}
early_access - {<class 'bool'>, <class 'float'>}
id - {<class 'str'>, <class 'float'>}
developer - {<class 'str'>, <class 'float'>}


Se elimina las columnas que no son relevantes para los objetivos del proyecto. Esto está desarrollado en una hoja de cálculo, en una matriz de objetivos vs. datos.

https://docs.google.com/spreadsheets/d/1ccOxyOM_yctvpk4-CbpX4kCihNn1rX1QifKjbtov10s/edit?usp=sharing

In [7]:
from transform import limpiar

limpiar.eliminar_columnas_innecesarias(
    df,
    [
        "publisher",
        "url",
        "tags",
        "reviews_url",
        "specs",
        "price",
        "early_access",
        "developer",
    ],
)
df.head()

,genres,app_name,title,release_date,id
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN


In [8]:
df.shape[0]

163147

Ahora, la cantidad de filas en el dataframe es de: 163147

## Variables
Las que utiliza este daframe son:

### Variables categóricas
* genres - variable categóricas nominales
* app_name - variable categórica nominal
* title - variable categórica nominal
* id - variable categórica nominal (identificador)

### Variables temporal
* release_date - variable temporal

## Manejando la columna "release_date"

Reviso la cantidad de registros de fecha que considero invalidas

In [9]:
df.head()

,genres,app_name,title,release_date,id
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN


In [10]:
from transform import info

info.celdas_vacias(df)

,Cantidad,Porcentaje
genres,91593,0.561414
app_name,88313,0.541309
title,90361,0.553862
release_date,90412,0.554175
id,88313,0.541309


In [11]:
def es_una_fecha_valida_uniformato(data, formato):
    from datetime import datetime

    try:
        datetime.strptime(str(data), formato)
        return True
    except ValueError:
        return False


def es_una_fecha_valida_multiformato(data, formatos):
    for i in formatos:
        if es_una_fecha_valida_uniformato(data, i):
            return True
    return False


def es_ignorado(data, formatos):
    for i in formatos:
        if str(data) == i:
            return True
    return False


count = 0
new_set = set()
formatos_a_ignorar = [
    "When it is finished",
    "Release Date TBA",
    "Coming soon..",
    "Demo is available now!",
    "Early Access soon",
    "coming soon",
    "nan",
    "Coming Soon",
    "Not yet available",
    "TBD",
    "TBA",
    "Soon..",
    "soon" "Coming soon",
    "Play Beta in demo!",
    "To be announced",
    "Demo coming soon.",
    "Coming soon",
    "Comming soon",
    "Comming Soon",
    "Coming Soon!"
    "SOON™",
    "SOON",
    "Soon™"
    "soon",
    "Soon",
    '"""Soon"""'
    "Coming Soon!",
    "Coming Soon/Próximamente",
    "When it is ready",
    "When it's done!",
    "When it's done",
    "Early Access Starting Soon!",
    "C'est bientôt...                    (ou pas)",
    "early access",
    "0̵1̴0̵0̶1̷0̶0̵0̴ ̴0̶0̶1̶1̶0̷0̶1̵1̴ ̸0̶0̶1̶1̵0̶1̷0̴0̵ ̴0̶1̷0̸1̵0̷0̴1̶0̴ ̴0̷0̴1̷1̶0̶1̵1̷1̵ ̵",
    "To Be Announced",
    "Please wait warmly",
    "Datachunks conflicted. Be vigiliant.",
    "To be Announced",
]

for i in df["release_date"]:
    if es_ignorado(i, formatos_a_ignorar):
        continue
    if not es_una_fecha_valida_multiformato(i, ["%Y-%m-%d", "%B %Y", "%Y", "%b %Y", "%d %b, %Y", "%d.%m.%Y"]):
        new_set.add(i)
        count += 1

print(f"Cantidad de registros con formato invalido de fecha: {count}")
print(list(new_set))

Cantidad de registros con formato invalido de fecha: 225
['14 July', 'Fall 2017', 'Late 2016 - Early 2017', 'TBA 2017', 'soon', '2018 [Now get free Pre Alpha]', 'Beta测试已开启', '2018年初頭発売予定', 'Spring 2017', 'SOON™', 'Январь 2018', 'Coming Soon!', '2018 early', 'Q2 2017', 'Q4 2017', 'Spring 2018', 'Summer 2017', 'Early 2018', 'End 2017', 'Coming Q3 2017', 'Early 2017', 'Hitting Early Access in 2017', 'Q1 (ish), 2017', 'Soon™', 'Q1, 2018', '"""Soon"""', 'Q1 2017', 'Early Spring 2018', "When it's done (2017)", '2017 Q1', 'Q1 2018', '1st Quarter 2018', 'Q2 2018 (Tentative)', '2018年1月', '2017 Q4', 'Coming late 2017', 'Winter 2017', '预热群52756441', 'First quarter of 2018', 'Fall 2018', 'Coming Fall 2017', 'Coming 2017', 'Coming Q1 2018', 'H2 2018', 'Q2 2018', 'Winter 2018']


Ahora, filtro y convierto el tipo de dato "release_date"

In [12]:
from datetime import datetime

def convertir_a_fecha(data):
    formatos = ["%Y-%m-%d", "%B %Y", "%Y", "%b %Y", "%d %b, %Y", "%d.%m.%Y"]
    for i in formatos:
        if es_una_fecha_valida_uniformato(data, i):
            return datetime.strptime(str(data), i)
    return None

df["release_date"] = df["release_date"].apply(convertir_a_fecha)

Verifico el filtro y el reemplazo los valores invalidos.

In [13]:
count = 0
new_set = set()

for i in df["release_date"]:
    if not isinstance(i, datetime):
        new_set.add(i)
        count += 1

print(f"Cantidad de registros con formato invalido de fecha: {count}")
print(list(new_set))

Cantidad de registros con formato invalido de fecha: 0
[]


Como los datos que hay en "release_date" son fechas completas, lo convertire en otro tipo de dato, pero solo utilizare el anio, no necesto ni el mes , ni el dia para las objetivos del proyecto.

In [14]:
from datetime import datetime
import pandas as pd

def obtener_fecha(fecha):
    if pd.isna(fecha):
        return fecha
    elif isinstance(fecha, datetime):
        return str(fecha.year)
    else:
        return None
    
df["release_date"] = df["release_date"].apply(obtener_fecha)

df.head()

,genres,app_name,title,release_date,id
0,NaN,NaN,NaN,NaT,NaN
1,NaN,NaN,NaN,NaT,NaN
2,NaN,NaN,NaN,NaT,NaN
3,NaN,NaN,NaN,NaT,NaN
4,NaN,NaN,NaN,NaT,NaN


## Manejando la columna "id"

In [15]:
from transform import normalizar

df = normalizar.cambiar_nombre_a_una_columna(df, "id", "item_id")

df.head()

,genres,app_name,title,release_date,item_id
0,NaN,NaN,NaN,NaT,NaN
1,NaN,NaN,NaN,NaT,NaN
2,NaN,NaN,NaN,NaT,NaN
3,NaN,NaN,NaN,NaT,NaN
4,NaN,NaN,NaN,NaT,NaN


## Manejando las columnas "app_name" y "title"

In [16]:
def combinar_columnas_de_texto(df, columna1, columna2, new_column):
    # Posibles registro
    # caso 1 - NaN NaN
    # caso 2 - str NaN
    # caso 3 - NaN str
    # caso 4 - str str
    def combinar_texto(row):
        if not isinstance(row[columna1], str):
            # ese condicion se ocupa del caso 1 y 3
            return row[columna2]
        elif not isinstance(row[columna2], str):
            # esta condicion se ocupa del caso 1 y 2
            return row[columna1]
        elif row[columna1] == row[columna2]:
            # esta condicion se ocupa del caso 4
            return row[columna1]
        else:
            # esta condicion se ocupa del caso 4
            return row[columna1] + " "+ row[columna2]

    df[new_column] = df.apply(combinar_texto, axis=1)
    df.drop(columns=[columna1, columna2], inplace=True)

combinar_columnas_de_texto(df, "app_name", "title", "name")

df.head()

,genres,release_date,item_id,name
0,NaN,NaT,NaN,NaN
1,NaN,NaT,NaN,NaN
2,NaN,NaT,NaN,NaN
3,NaN,NaT,NaN,NaN
4,NaN,NaT,NaN,NaN


## Limpieza

Revisamos las celdas vacias en el dataframe.

In [17]:
from transform import info

info.celdas_vacias(df)

,Cantidad,Porcentaje
genres,91593,0.561414
release_date,90902,0.557178
item_id,88313,0.541309
name,88313,0.541309


Como la columna "item_id" es un identificador, no deberia tener valores nulos

In [18]:
from transform import limpiar

limpiar.eliminar_filas_por_celdas_vacias_en(df, ["item_id"])

la cantidad de filas en el dataframe es: 163147
despues de eliminar las filas vacias es: 74834


In [19]:
from transform import limpiar

limpiar.eliminar_filas_vacias(df)

la cantidad de filas en el dataframe es: 74834
despues de eliminar las filas vacias es: 74834


In [20]:
from transform import limpiar

df = limpiar.eliminar_filas_duplicadas(df)

la cantidad de filas en el dataframe es: 74834
despues de eliminar las filas repetidas es: 74833


Volvemos a revisar las celdas vacias en el dataframe

In [21]:
from transform import info

info.celdas_vacias(df)

,Cantidad,Porcentaje
genres,3282,0.043858
release_date,2591,0.034624
item_id,0,0.000000
name,2,0.000027


los datos nulos en las columnas "genres", "release_date", "name" no pueden ser manejados de manera efectiva. Lo que podría afectar la calidad de los resultados. Asi que, se eliminara las filas donde exista registros NaN.

In [22]:
from transform import limpiar

limpiar.eliminar_filas_por_celdas_vacias_en(df, ["genres", "release_date","name"])

la cantidad de filas en el dataframe es: 74833
despues de eliminar las filas vacias es: 71006


In [23]:
from transform import info

info.celdas_vacias(df)

,Cantidad,Porcentaje
genres,0,0.0
release_date,0,0.0
item_id,0,0.0
name,0,0.0


In [24]:
df.shape[0]

71006

Ahora, la cantidad de filas en el dataframe es de: 71006

Ahora reorganizaré las columnas para apreciar los datos de una manera más sencilla

In [25]:
df.head()

,genres,release_date,item_id,name
88310,Action,2018,761140,Lost Summoner Kitty
88311,Casual,2018,761140,Lost Summoner Kitty
88312,Indie,2018,761140,Lost Summoner Kitty
88313,Simulation,2018,761140,Lost Summoner Kitty
88314,Strategy,2018,761140,Lost Summoner Kitty


In [26]:
df = df[["item_id", "name", "release_date", "genres"]]

df.head()

,item_id,name,release_date,genres
88310,761140,Lost Summoner Kitty,2018,Action
88311,761140,Lost Summoner Kitty,2018,Casual
88312,761140,Lost Summoner Kitty,2018,Indie
88313,761140,Lost Summoner Kitty,2018,Simulation
88314,761140,Lost Summoner Kitty,2018,Strategy


## Load

In [27]:
#df_steam_game.to_parquet("..\\data\\intermediate\\steam_games.parquet")
df.to_parquet("../data/intermediate/steam_games.parquet")